# KS Distance Minimization for Low $\mu_2$ Values

In [ ]:
def plot_M_SVMM(X, n, pi, alpha, lambda_, mu, sigma):
    ax = plt.subplots(figsize = (6,6))
    plt.hist(X, alpha = 0.20, bins = max(X), color = 'grey', edgecolor = 'white', linewidth = 3) # plot histogram of input data set

    curve = np.linspace(mu - 10 * sigma[0], mu + 10 * sigma[0], 1000)
    curve2 = np.linspace(1, 10 * lambda_, 1000)
    plt.plot(curve, n * norm.pdf(curve, mu, sigma[0]) * pi[1], linewidth = 3, color='red')
    plt.plot(curve2, n * expon.pdf(curve2, loc = 0, scale = lambda_) * pi[0], linewidth = 3, color = 'red')
    plt.scatter(0, n * alpha * pi[0], color = 'red')

    ax.set_yscale("log") 
    ax.set_ylim(bottom=1)
    ax.set_xlim(left=0, right = max(X)+1)
    plt.xlabel('Evidence Depth')
    plt.ylabel('Number of Samples')
    plt.show()

In [ ]:
def M_SVMM(X, mu_2, sigma_3, show_plot, show_distance):

    # TODO: fine-tune initial paramters, put in separate function
    # initial values for parameters
    n = np.shape(X)[0] # length of data set
    pi = [1.0/3 for _ in range(3)] # mixing coefficients
    r = np.zeros([3,n]) # responsibility matrix
    alpha = np.sum(X == 0) / n # probability of an observation belonging to zero component
    lambda_ = n / np.sum(X != 0) # expected value of the geometric distribution
    sigma = [np.std(X), sigma_3] # standard deviation for normal distribution
    mu = [mu_2, 2*mu_2] # fixed values for mu_2 and 2mu_2
    log_likelihoods = [] 
    iteration = 0
    distance = 1

    while distance > (1/(n*10)): # convergence criterion

        # expectation
        r[0][X==0] = pi[0] * alpha # responsibility of zero values to first mode
        r[0][X!=0] = pi[0] * (1-alpha) * geom.pmf(X[X!=0], 1/lambda_) # responsibility of nonzero values to first component
        r[1] = pi[1] * norm.pdf(X, mu[0], sigma[0]) # responsibility of each value to second component
        r[2] = pi[2] * norm.pdf(X, mu[1], sigma[1]) # responsibility of each value to second component
        r = r / np.sum(r, axis = 0) # normalization

        # maximization
        pi = np.sum(r, axis = 1) / n # total responsibility of a component divided by total # of observations
        alpha = np.sum(r[0][X==0]) / np.sum(r[0]) # MLE for 
        lambda_ = np.dot(X[X != 0], r[0][X != 0]) / ((1-alpha)*np.sum(r[0])) # reciprocal of MLE for p in geometric distribution
        # mu[0] = np.average(X, weights = r[1]) # MLE for mean in normal distribution
        # mu[1] = np.average(X, weights = r[2]) # MLE for mean in normal distribution
        sigma[0] = np.average((X-mu[0])**2, weights=r[1])**.5 # MLE for standard deviation in normal distribution
        # sigma[1] = np.average((X-mu[1])**2, weights=r[2])**.5 # MLE for standard deviation in normal distribution

        # score
        hurdle = np.where(X == 0, pi[0] * alpha, pi[0] * (1-alpha) * geom.pmf(X,1/lambda_)) #  likelihood of each observation in hurdle model
        gmm = pi[1] * norm.pdf(X, mu[0], sigma[0]) +  pi[2] * norm.pdf(X, mu[1], sigma[1])# likelihood of each observation in normal distribution
        log_likelihood = np.sum(np.log(hurdle + gmm)) # sum of log of likelihood of each observation
        log_likelihoods.append(log_likelihood) 

        iteration += 1
        if iteration > 1:
            distance = np.abs(log_likelihoods[-2]-log_likelihoods[-1]) # magnitude of difference between each 
            if(show_distance):
                print(distance)

    if(show_plot):
        plot_M_SVMM(X, n, pi, alpha, lambda_, mu, sigma)

    return pi, alpha, lambda_, mu, sigma

In [ ]:
def norm_samples(samples, mu2_times2, sigma_3, show_plot):
    
    CDF_x = truncnorm.cdf(np.sort(samples), a=0, b=np.inf, loc=mu2_times2, scale=sigma_3)
    edf_vals = np.arange(1,len(samples)+1)/float(len(samples))

    if show_plot:
        plt.plot(np.sort(samples), CDF_x, label = 'trunc_cdf', color = 'red')
        plt.plot(np.sort(samples), edf_vals, label = 'edf', color = 'k', linestyle = '--')
        plt.legend(loc = 'lower right')
        plt.xlabel('X')
        plt.ylabel('Probability')
        plt.show()

    #d, p_val = ks_2samp(np.sort(norm_samples), np.sort(samples))

    # KS DISTANCE BY HAND
    diffs = [abs(cdf_val - edf_val) for cdf_val, edf_val in zip(CDF_x, edf_vals)]
    ks_distance_hand = max(diffs)
    d = ks_distance_hand
    if show_plot:
        print('d: ', d, 'd_by_hand:', ks_distance_hand)

    return d, CDF_x

In [ ]:
def ks_2mu2(X_gen, Y_gen, mu2_times2, sigma_3):

    samples = [x for x in X_gen if x >= mu2_times2] # values where xi >= 2mu_2
    edf_vals = np.arange(1,len(samples)+1)/float(len(samples)) # fix the list

    # d_range = []
    # trunc_cdf_sets = []
    # for _ in range(10):
    #     d, CDF_x = norm_samples(samples, mu2_times2, sigma_3, False)
    #     d_range.append(d)
    #     trunc_cdf_sets.append(CDF_x)

    # d = np.average(d_range)
    # CDF_x = np.average(trunc_cdf_sets, axis = 0)

    d, CDF_x = norm_samples(samples, mu2_times2, sigma_3, False)

    return d, CDF_x, edf_vals, samples

# TODO: create wrapper for norm_samples to get average D from 5 runs, do smoothing after comfirming the parabola

In [ ]:
mus = []
for _ in range(10):
    X_gen = theta_to_data(N, pi, alpha, lambda_, 10, sigma)
    theta = SVMM(X_gen, False, False) # estimated theta
    mu2_times2 = theta[3][1] # 2mu_2
    mus.append(mu2_times2)
mu2_times2 = np.average(mus)
mu2times2hat_range = np.linspace(mu2_times2 - 3, mu2_times2 + 3, 40)
print(mu2times2hat_range)

# d_range = []
# for delta in mu2times2hat_range:
#     ds = []
#     for _ in range(10):
#         X_gen = theta_to_data(N, pi, alpha, lambda_, delta, [sigma[0],sigma_3])
#         theta = SVMM(X_gen, False, False) # estimated theta
#         sigma_3 = theta[4][1]
#         d, CDF_x, edf_vals, samples = ks_2mu2(X_gen, delta, sigma_3)
#         ds.append(d)
#     d = np.average(ds)
#     d_range.append(d)
#     # trunc_cdf_sets.append(CDF_x)
#     # edf_sets.append(edf_vals)
#     # samples_list.append(samples)

# plt.plot(mu2times2hat_range, d_range, '-o', color = 'purple')
# plt.axvline(mu2times2hat_range[d_range.index(min(d_range))], color='red')
# plt.xlabel(r'$2\mu_2$')
# plt.ylabel('KS distance')
# plt.show()

In [ ]:
# Y_gen and EDF changes with delta
def ks_distance_svmm(X_gen, show_plot, refine_factor):
    theta = SVMM(X_gen, False, False) 
    mu2_times2 = theta[3][1] 
    sigma_3 = theta[4][1]
    mu2times2hat_range = np.linspace(mu2_times2 - 5, mu2_times2 + 5, 40)
    d_range_squared = []
    for _ in range(refine_factor):
        d_range = []
        for delta in mu2times2hat_range:
            Y_gen = [x for x in X_gen if x >= delta] 
            EDF_y = np.arange(1,len(Y_gen)+1)/float(len(Y_gen))
            X_delta = X_rvs_left(delta, sigma_3, len(Y_gen), mu2_times2)
            CDF_x = truncnorm.cdf(np.sort(X_delta), a=0, b=np.inf, loc=mu2_times2, scale=sigma_3)
            diffs = [abs(cdf_val - edf_val) for cdf_val, edf_val in zip(CDF_x, EDF_y)]
            d_range.append(max(diffs))
        d_range_squared.append(d_range)
    d_range = np.average(d_range_squared, axis=0)
    best_mu = mu2times2hat_range[np.where(d_range == (min(d_range)))]
    print(mu2_times2,',',best_mu)

    # PLOT D AGAINST 2MU_2
    if show_plot:
        plt.plot(mu2times2hat_range, d_range, '-o', color = 'purple')
        plt.axvline(best_mu, color='red')
        plt.xlabel(r'$2\mu_2$')
        plt.ylabel('KS distance')
        plt.show()
for mu2 in range(5,10):
    print(mu2)
    X_gen = theta_to_data(N, pi, alpha, lambda_, mu2, [5,5]) 
    ks_distance_svmm(X_gen, True, 10)

# Error Function; Determining Expected # of Zeroes and Actual Number of Zeroes

In [ ]:
def theta_to_data2(N, pi, alpha, lambda_, mu_2, sigma):

    draws = multinomial.rvs(n = N, p = pi)
    draws_alpha = multinomial.rvs(n = draws[0], p = [alpha, 1 - alpha])

    X = [0 for _ in range(draws_alpha[0])]
    X_l = geom.rvs(1/lambda_, size = draws_alpha[1])
    X_g = norm.rvs(mu_2, sigma[0], size = draws[1])
    X_g2 = norm.rvs(2 * mu_2, sigma[1], size = draws[2])

    while sum(X_g < -0.5) > 0:
        X_g_new = norm.rvs(mu_2, sigma[0], size = sum(X_g < -0.5))
        X_g = [x for x in X_g if x >= -0.5]
        X_g = np.concatenate((X_g, X_g_new))
    X_g = np.round(X_g).astype(int)
    zeros_mode2 = sum(X_g == 0)

    while sum(X_g2 < -0.5) > 0:
        X_g2_new = norm.rvs(2 * mu_2, sigma[1], size = sum(X_g2 < -0.5))
        X_g2 = [x for x in X_g2 if x >= -0.5]
        X_g2 = np.concatenate((X_g2, X_g2_new))
    X_g2 = np.round(X_g2).astype(int)

    X = np.concatenate((X,X_l))
    X = np.concatenate((X,X_g))
    X = np.concatenate((X,X_g2))

    return X, zeros_mode2

In [ ]:
def single_run(N, pi, alpha, lambda_, mu, sigma):
    X_gen, zeros = theta_to_data2(N, pi, alpha, lambda_, mu, sigma)
    mu_2_hat = SVMM(X_gen, False, False)[3][0]
    e_zero_mode2 = N * pi[1] * ((norm.cdf(0.5, loc = mu, scale = sigma[0])-norm.cdf(-0.5, loc = mu, scale = sigma[0]))/(1-norm.cdf(-0.5, loc = mu, scale = sigma[0])))
    return [mu_2_hat, e_zero_mode2, zeros]

def plot_mu(N, pi, alpha, lambda_, sigma, start, stop, step):
    mus = np.arange(start, stop, step)
    num_runs = 10
    ests_mu2 = []
    ests_e_mode2 = []
    avg_zeros = []
    
    for mu in mus:
        ests = Parallel(n_jobs=-1)(delayed(single_run)(N, pi, alpha, lambda_, mu, sigma) for _ in range(num_runs))
        ests_mu2.append(np.mean(ests[0][0]))
        ests_e_mode2.append(np.mean(ests[0][1]))
        avg_zeros.append(np.mean(ests[0][2]))
        print('e',ests)
    return ests_mu2, ests_e_mode2, avg_zeros

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)

ests_mu2, ests_e_mode2, avg_zeros = plot_mu(N, pi, alpha, lambda_, sigma, 1,25,1)
plt.scatter(ests_mu2, ests_e_mode2, marker = 'v',color = 'green', alpha = 0.5, label = r'expected', zorder = 4)
plt.scatter(ests_mu2, avg_zeros, marker = 'v', color = 'red', alpha = 0.5, label = r'actual', zorder = 4)

ax.set_xlabel(r'$\hat{\mu}_2$')
ax.set_ylabel(r'#0s')
ax.set_ylim(bottom=0)
ax.set_xlim(left=0)
leg = plt.legend(loc = 'upper right', fontsize = 'small', frameon = False, title = r'$n = 1000, \pi = [\frac{1}{3},\frac{1}{3},\frac{1}{3}],\mu s = [1,25,1]$')
leg._legend_box.align = "left"
plt.show()

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)

ests_mu2, ests_e_mode2, avg_zeros = plot_mu(N, pi, alpha, lambda_, sigma, 1.2, 30, 1.2)
plt.scatter(ests_mu2, ests_e_mode2, marker = '.', s = 100, alpha = 0.5, color = 'green',label = r'expected', zorder = 3)
plt.scatter(ests_mu2, avg_zeros, marker = '.', s = 100, alpha = 0.5, color = 'red',label = r'actual', zorder = 3)
ax.set_xlabel(r'$\hat{\mu}_2$')
ax.set_ylabel(r'#0s')
ax.set_ylim(bottom=0)
ax.set_xlim(left=0)
leg = plt.legend(loc = 'upper right', fontsize = 'small', frameon = False, title = r'$n = 1000, \pi = [\frac{1}{3},\frac{1}{3},\frac{1}{3}],\mu s = [1.2,30,1.2]$')
leg._legend_box.align = "left"
plt.show()

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)

ests_mu2, ests_e_mode2, avg_zeros = plot_mu(N, pi, alpha, lambda_, sigma, 1.4 , 35, 1.4)
plt.scatter(ests_mu2, ests_e_mode2, marker = '^', color = 'green',alpha = 0.5,  label = r'expected', zorder = 2)
plt.scatter(ests_mu2, avg_zeros, marker = '^', color = 'red', alpha = 0.5, label = r'actual', zorder = 2)
ax.set_xlabel(r'$\hat{\mu}_2$')
ax.set_ylabel(r'#0s')
ax.set_ylim(bottom=0)
ax.set_xlim(left=0)
leg = plt.legend(loc = 'upper right', fontsize = 'small', frameon = False, title = r'$n = 1000, \pi = [\frac{1}{3},\frac{1}{3},\frac{1}{3}], \mu s = [1.4,35,1.4]$')
leg._legend_box.align = "left"
plt.show()

In [ ]:
def single_run(N, pi, alpha, lambda_, mu, sigma):
    X_gen = theta_to_data(N, pi, alpha, lambda_, mu, sigma)
    mu_2_hat = SVMM(X_gen, False, False)[3][0]
    e_zero_mode2 = N * pi[1] * ((norm.cdf(0.5, loc = mu, scale = sigma[0])-norm.cdf(-0.5, loc = mu, scale = sigma[0]))/(1-norm.cdf(-0.5, loc = mu, scale = sigma[0])))
    return [mu_2_hat, e_zero_mode2]

def plot_mu(N, pi, alpha, lambda_, sigma, start, stop, step):
    mus = np.arange(start, stop, step)
    num_runs = 10
    ests_mae = []
    ests_e_mode2 = []
    
    for mu in mus:
        ests = Parallel(n_jobs=-1)(delayed(single_run)(N, pi, alpha, lambda_, mu, sigma) for _ in range(num_runs))
        ests_mae.append(np.mean(np.abs(ests[0] - mu)))
        ests_e_mode2.append(np.mean(ests[1]))

    return ests_mae, ests_e_mode2

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)

ests_mae, ests_e_mode2 = plot_mu(N, pi, alpha, lambda_, sigma, 1,25,1)
plt.scatter(ests_e_mode2, ests_mae, marker = 'v', color = '#6600CC', label = r'$\mu s= [1,25,1]$', zorder = 4)

ests_mae, ests_e_mode2 = plot_mu(N, pi, alpha, lambda_, sigma, 1.2, 30, 1.2)
plt.scatter(ests_e_mode2, ests_mae, marker = '.', s = 100,color = '#9933FF',label = r'$\mu s = [1.2,30,1.2]$', zorder = 3)

ests_mae, ests_e_mode2 = plot_mu(N, pi, alpha, lambda_, sigma, 1.4 , 35, 1.4)
plt.scatter(ests_e_mode2, ests_mae, marker = '^', color = '#CC99FF', label = r'$\mu s = [1.4,35,1.4]$', zorder = 2)

ax.set_xlabel(r'$E[0 | mode_2]$')
ax.set_ylabel(r'MAE')
ax.set_ylim(bottom=0)
ax.set_xlim(left=0, right = 35)
plt.legend(loc = 'upper right', fontsize = 'small', frameon = False, title = r'$n = 1000, \pi = [\frac{1}{3},\frac{1}{3},\frac{1}{3}]$')
plt.show()

In [ ]:
for index in diff_rows.index:
    print('original')
    sample_nonzero = depths_nonzero.iloc[index]
    pi, alpha, lambda_, mu, sigma = SVMM(sample_nonzero, True, False)

    print('pi:', pi)
    print('alpha:', alpha)
    print('lambda:', lambda_)
    print('mu:', mu)
    print('sigma:', sigma)

    print('no outliers')
    sample_no_outlier = depths_no_outlier.iloc[index]
    pi, alpha, lambda_, mu, sigma = SVMM(sample_no_outlier, True, False)

    print('pi:', pi)
    print('alpha:', alpha)
    print('lambda:', lambda_)
    print('mu:', mu)
    print('sigma:', sigma)